# <B> # SageMaker Training with Experiments and Processing </B>
* Container: codna_python3

## 학습 작업의 실행 노트북 개요

- SageMaker Training에 SageMaker 실험을 추가하여 여러 실험의 결과를 비교할 수 있습니다.
    - [작업 실행 시 필요 라이브러리 import](#작업-실행-시-필요-라이브러리-import)
    - [SageMaker 세션과 Role, 사용 버킷 정의](#SageMaker-세션과-Role,-사용-버킷-정의)
    - [하이퍼파라미터 정의](#하이퍼파라미터-정의)
    - [학습 실행 작업 정의](#학습-실행-작업-정의)
        - 학습 코드 명
        - 학습 코드 폴더 명
        - 학습 코드가 사용한 Framework 종류, 버전 등
        - 학습 인스턴스 타입과 개수
        - SageMaker 세션
        - 학습 작업 하이퍼파라미터 정의
        - 학습 작업 산출물 관련 S3 버킷 설정 등
    - [학습 데이터셋 지정](#학습-데이터셋-지정)
        - 학습에 사용하는 데이터셋의 S3 URI 지정
    - [SageMaker 실험 설정](#SageMaker-실험-설정)
    - [학습 실행](#학습-실행)
    - [데이터 세트 설명](#데이터-세트-설명)
    - [실험 결과 보기](#실험-결과-보기)

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 0. Install packages

In [32]:
install_needed = True  # should only be True once
# install_needed = False

In [33]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

Already revised


In [14]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U smdebug sagemaker-experiments
    !{sys.executable} -m pip install -U sagemaker
    
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 81.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached smdebug-1.0.12-py2.py3-none-any.whl (270 kB)
  Using cached sagemaker_experiments-0.1.43-py3-none-any.whl (42 kB)
  Using cached pyinstrument-3.4.2-py2.py3-none-any.whl (83 kB)
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached sagemaker-2.146.0.tar.gz (718 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.146.0-py2.py3-none-any.whl size=964936 sha256=d76ac45fe0127ff8834f5d6f7b001cab80f2acdc66ff67ae381644180f04f068
  Stored in directory: /home/ec2-user/.cach

## 1. parameter store 설정

In [21]:
import boto3
from utils.ssm import parameter_store

In [22]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

In [23]:
strBucketName = pm.get_params(key="-".join([strPrefix, "BUCKET"]))
strExecutionRole = pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]))

In [24]:
print (f'strBucketName: {strBucketName}')
print (f'strExecutionRole: {strExecutionRole}')

strBucketName: sagemaker-us-east-1-419974056037
strExecutionRole: arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436


## 2. Dataset

In [34]:
import os

In [35]:
strS3DataPath = f"s3://{strBucketName}/dataset" 
strLocalDataPath = os.path.join(os.getcwd(), "data")

In [36]:
!aws s3 sync $strLocalDataPath $strS3DataPath

## 3.Training-job

In [50]:
import os
import sagemaker
from sagemaker.xgboost.estimator import XGBoost

* **Set Up SageMaker Experiment**
    - Create or load [SageMaker Experiment](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) for the example training job. This will create an experiment trial object in SageMaker.

In [51]:
from time import strftime
from smexperiments.trial import Trial
from smexperiments.experiment import Experiment

In [52]:
def create_experiment(experiment_name):
    try: sm_experiment = Experiment.load(experiment_name)
    except: sm_experiment = Experiment.create(experiment_name=experiment_name)

In [53]:
def create_trial(experiment_name):
    create_date = strftime("%m%d-%H%M%s")
    sm_trial = Trial.create(trial_name=f'{experiment_name}-{create_date}',
                            experiment_name=experiment_name)
    job_name = f'{sm_trial.trial_name}'
    return job_name

* params for training job

In [54]:
# Set to True to enable SageMaker to run locally
local_mode = True

if local_mode:
    
    from sagemaker.local import LocalSession
    
    strInstanceType = "local"
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
        
    dicDataChannels = {
        "training": f'file://{os.path.join(strLocalDataPath, "train.csv")}',
        "testing": f'file://{os.path.join(strLocalDataPath, "test.csv")}',
    }
    
else:
    strInstanceType = "ml.m5.2xlarge" #"ml.p3.2xlarge"#"ml.g4dn.8xlarge"#"ml.p3.2xlarge", 'ml.p3.16xlarge' , ml.g4dn.8xlarge
    
    sagemaker_session = sagemaker.Session()
    dicDataChannels = {
        "training": os.path.join(strS3DataPath, "train.csv"), 
        "testing": os.path.join(strS3DataPath, "test.csv"), 
    }

nInstanceCount = 1

bSpotTraining = False
if bSpotTraining:
    nMaxWait = 1*60*60
    nMaxRun = 1*60*60
    
else:
    nMaxWait = None
    nMaxRun = 1*60*60
    

bUseTrainWarmPool = False ## training image 다운받지 않음, 속도 빨라진다
if bUseTrainWarmPool: nKeepAliveSeconds = 3600 ## 최대 1시간 동안!!, service quota에서 warmpool을 위한 request 필요
else: nKeepAliveSeconds = None
if bSpotTraining:
    bUseTrainWarmPool = False # warmpool은 spot instance 사용시 활용 할 수 없음
    nKeepAliveSeconds = None
    


strOutputPath = os.path.join(
    "s3://{}".format(strBucketName),
    strPrefix,
    "training",
    "model-output"
)

strCodeLocation = os.path.join(
    "s3://{}".format(strBucketName),
    strPrefix,
    "training",
    "backup_codes"
)

strExperimentName = '-'.join([strPrefix, "experiments"])

## You can't override the metric definitions for Amazon SageMaker algorithms. 
# strNumeticRegEx = "([0-9\\.]+)(e-?[[01][0-9])?"
# listMetricDefinitions = [
#     {"Name": "train_loss", "Regex": f"loss={strNumeticRegEx}"},
#     {"Name": "wer", "Regex": f"wer:{strNumeticRegEx}"}
# ]

# dicGitConfig = {
#     'repo': f'https://{pm.get_params(key="-".join([prefix, "CODE_REPO"]))}',
#     'branch': 'main',
#     'username': pm.get_params(key="-".join([prefix, "CODECOMMIT-USERNAME"]), enc=True),
#     'password': pm.get_params(key="-".join([prefix, "CODECOMMIT-PWD"]), enc=True)
# }  

kwargs = {}

In [55]:
print (f'strInstanceType: {strInstanceType}')
print (f'nInstanceCount: {nInstanceCount}')
print (f'sagemaker_session: {sagemaker_session}')
print (f'bSpotTraining: {bSpotTraining}')
print (f'strExperimentName: {strExperimentName}')
print (f'dicDataChannels: {dicDataChannels}')
print (f'strOutputPath: {strOutputPath}')
print (f'strCodeLocation: {strCodeLocation}')
print (f'bUseTrainWarmPool: {bUseTrainWarmPool}/{nKeepAliveSeconds}')

strInstanceType: local
nInstanceCount: 1
sagemaker_session: <sagemaker.local.local_session.LocalSession object at 0x7fce158befe0>
bSpotTraining: False
strExperimentName: DJ-SM-IMD-experiments
dicDataChannels: {'training': 'file:///home/ec2-user/SageMaker/sagemaker-immersion-day/data/train.csv', 'testing': 'file:///home/ec2-user/SageMaker/sagemaker-immersion-day/data/test.csv'}
strOutputPath: s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD/training/model-output
strCodeLocation: s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD/training/backup_codes
bUseTrainWarmPool: False/None


* Define training job

In [56]:
dicHyperparameters = {
    "scale_pos_weight" : "19",    
    "max_depth": "2",
    "eta": "0.3",
    "objective": "binary:logistic",
    "num_round": "100",
}

In [57]:
estimator = XGBoost(
    entry_point="xgboost_starter_script.py",
    source_dir="source/train/",
    output_path=strOutputPath,
    code_location=strCodeLocation,
    hyperparameters=dicHyperparameters, ## Contatiner내 env. variable로 들어 감
    role=strExecutionRole,
    sagemaker_session=sagemaker_session,
    instance_count=nInstanceCount,
    instance_type=strInstanceType,
    framework_version="1.3-1",
    max_run=nMaxRun,
    use_spot_instances=bSpotTraining,
    max_wait=nMaxWait,
    keep_alive_period_in_seconds=nKeepAliveSeconds,
    enable_sagemaker_metrics=True,
    #metric_definitions=listMetricDefinitions,
    volume_size=256, ## GB
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: local.


* run

In [58]:
if strInstanceType =='local_gpu': estimator.checkpoint_s3_uri = None

create_experiment(strExperimentName)
job_name = create_trial(strExperimentName)

estimator.fit(
    inputs=dicDataChannels, 
    job_name=job_name,
    experiment_config={
      'TrialName': job_name,
      'TrialComponentDisplayName': job_name,
    },
    wait=True,
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating training-job with name: DJ-SM-IMD-experiments-0424-07451682322339
INFO:sagemaker.local.local_session:Starting training job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-177j7:
    command: train
    container_name: m54u956j17-algo-1-177j7
    environment:
    - '[Masked]'
    - '[Masked]'
    image: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1
    networks:
      sagemaker

Creating m54u956j17-algo-1-177j7 ... 
Creating m54u956j17-algo-1-177j7 ... done
Attaching to m54u956j17-algo-1-177j7
m54u956j17-algo-1-177j7 | [2023-04-24 07:45:42.777 3fd79362b216:1 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
m54u956j17-algo-1-177j7 | [2023-04-24 07:45:42.807 3fd79362b216:1 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
m54u956j17-algo-1-177j7 | [2023-04-24:07:45:42:INFO] Imported framework sagemaker_xgboost_container.training
m54u956j17-algo-1-177j7 | [2023-04-24:07:45:42:INFO] No GPUs detected (normal if no gpus installed)
m54u956j17-algo-1-177j7 | [2023-04-24:07:45:42:INFO] Invoking user training script.
m54u956j17-algo-1-177j7 | [2023-04-24:07:45:42:INFO] Installing module with the following command:
m54u956j17-algo-1-177j7 | /miniconda3/bin/python3 -m pip install . 
m54u956j17-algo-1-177j7 | Processing /opt/ml/code
m54u956j17-algo-1-177j7 |   Preparing metadata (setup.py) ... 

INFO:root:creating /tmp/tmp94v95g9j/artifacts/output/data
INFO:root:copying /tmp/tmp94v95g9j/algo-1-177j7/output/data/metrics.json -> /tmp/tmp94v95g9j/artifacts/output/data
INFO:root:copying /tmp/tmp94v95g9j/model/xgboost-model -> /tmp/tmp94v95g9j/artifacts/model


m54u956j17-algo-1-177j7 exited with code 0
Aborting on container exit...


===== Job Complete =====


* save model-path, experiment-name

In [18]:
pm.put_params(key="-".join([strPrefix, "MODEL-PATH"]), value=estimator.model_data, overwrite=True)
pm.put_params(key="-".join([strPrefix, "EXPERI-NAME"]), value=strExperimentName, overwrite=True)

'Store suceess'

* show experiments

In [19]:
from sagemaker.analytics import ExperimentAnalytics
import pandas as pd
#pd.options.display.max_columns = 50
#pd.options.display.max_rows = 10
#pd.options.display.max_colwidth = 100

In [20]:
trial_component_training_analytics = ExperimentAnalytics(
    sagemaker_session= sagemaker_session,
    experiment_name= strExperimentName,
    sort_by="metrics.validation:auc.max",        
    sort_order="Descending",
    metric_names=["validation:auc"]
)

trial_component_training_analytics.dataframe()[['Experiments', 'Trials', 'validation:auc - Min', 'validation:auc - Max',
                                                'validation:auc - Avg', 'validation:auc - StdDev', 'validation:auc - Last', 
                                                'eta', 'max_depth', 'num_round', 'scale_pos_weight']]

,Experiments,Trials,validation:auc - Min,validation:auc - Max,validation:auc - Avg,validation:auc - StdDev,validation:auc - Last,eta,max_depth,num_round,scale_pos_weight
0,[DJ-SM-IMD-experiments],[DJ-SM-IMD-experiments-0424-04371682311053],1.000000,1.000000,1.000000,0.0,1.000000,"""0.3""","""2""","""100""","""19"""
1,[DJ-SM-IMD-experiments],[DJ-SM-IMD-experiments-0424-04281682310513],1.000000,1.000000,1.000000,0.0,1.000000,"""0.3""","""2""","""100""","""19"""
2,[DJ-SM-IMD-experiments],[DJ-SM-IMD-experiments-0412-10121681294361],0.821124,0.821124,0.821124,0.0,0.821124,"""0.3""","""2""","""100""","""19"""
3,[DJ-SM-IMD-experiments],[DJ-SM-IMD-experiments-0416-06421681627343],0.821124,0.821124,0.821124,0.0,0.821124,"""0.3""","""2""","""100""","""19"""
4,[DJ-SM-IMD-experiments],[DJ-SM-IMD-experiments-0419-04191681877971],0.821124,0.821124,0.821124,0.0,0.821124,"""0.3""","""2""","""100""","""19"""
